In [1]:
%pip install ultralytics > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ultralytics
import torch
import os
import json
import yaml
import pandas as pd
import shutil

In [3]:
os.listdir('/kaggle/input')

['urban-issues-image-classification']

In [4]:
train = pd.read_csv('/kaggle/input/urban-issues-image-classification/train.csv')
test = pd.read_csv('/kaggle/input/urban-issues-image-classification/test.csv')


In [5]:
os.mkdir('/kaggle/working/dataset')
os.mkdir('/kaggle/working/dataset/train')
os.mkdir('/kaggle/working/dataset/test')

for i in range(1, 7):
    os.mkdir(f'/kaggle/working/dataset/train/{i}')
    os.mkdir(f'/kaggle/working/dataset/test/{i}')



In [6]:
fromDir = '/kaggle/input/urban-issues-image-classification/train/'

for row in train.itertuples(): # idx2=filename, idx3=class
    shutil.copyfile(fromDir + row[2], f'/kaggle/working/dataset/train/{row[3]}/{row[2]}')
    shutil.copyfile(fromDir + row[2], f'/kaggle/working/dataset/test/{row[3]}/{row[2]}')

In [7]:
# total = 0
# for dir in os.listdir('/kaggle/working/dataset/train/'):
#     total += len(os.listdir(f'/kaggle/working/dataset/train/{dir}'))
#     print(len(os.listdir(f'/kaggle/working/dataset/train/{dir}')))

In [8]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")

GPU is available


In [9]:
model = ultralytics.YOLO('yolov8l-cls.pt')

model.to(device)

100%|██████████| 71.7M/71.7M [00:00<00:00, 297MB/s]


YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

In [10]:
# Currently not using wandb
os.environ['WANDB_DISABLED'] = 'true'

In [11]:
model.train(data='/kaggle/working/dataset', epochs=20, imgsz=640)

engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/kaggle/working/dataset, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8

2024-05-10 09:46:38,176	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-10 09:46:38,920	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-10 09:46:40.917581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 09:46:40.917682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 09:46:41.030856: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLA

Overriding model.yaml nc=1000 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   4720640  ultralyti

wandb: Tracking run with wandb version 0.16.6
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...


20 epochs completed in 0.483 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train/weights/best.pt, 72.6MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.11 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8l-cls summary (fused): 133 layers, 36192262 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /kaggle/working/dataset/train... found 1708 images in 6 classes ✅ 
val: None...
test: /kaggle/working/dataset/test... found 1708 images in 6 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 54/54 [00:22<00:00,  2.42it/s]


                   all      0.999          1
Speed: 0.6ms preprocess, 10.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
wandb:                  lr/pg1 ▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
wandb:                  lr/pg2 ▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
wandb:   metrics/accuracy_top1 ▁▃▃▄▄▅▆▇▇███████████
wandb:   metrics/accuracy_top5 ▁▆▆█▇▇██████████████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:              train/loss █▆▆▅▅▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁
wandb:                val/loss █▆▆▅▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 4e-05
wandb:                  lr/pg1 4e-05
wandb:                  lr/pg2 4e-05
wandb:   metrics/accuracy_top1 0.99883
wandb:   metrics/accuracy_top5 1.0
wandb:            model/GFLOPs 99.129
wandb:        model/parameters 36207430
wandb: model/speed_PyTorch(ms) 9.905
wandb:              train/loss 0.07121
wandb:                val/loss 1.04785
wandb: 
wandb: Y

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3efcb64c40>
curves: []
curves_results: []
fitness: 0.9994145333766937
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9988290667533875, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9994145333766937}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.5820244201731627, 'inference': 10.370277967609342, 'loss': 0.000682731422942472, 'postprocess': 0.0005731537973015314}
task: 'classify'
top1: 0.9988290667533875
top5: 1.0

In [12]:
os.mkdir('/kaggle/working/predimg')

In [13]:
fromDir = '/kaggle/input/urban-issues-image-classification/test/'
for img in os.listdir(fromDir):
    shutil.copy(fromDir + img, f'/kaggle/working/predimg/{img}')

In [14]:
ids = []
classpred = []

for row in test.itertuples():
    ids.append(row[1])
    filename = row[2]
    prediction = model(f'/kaggle/working/predimg/{filename}')
    classpred.append(prediction[0].names[int(prediction[0].probs.data.argmax().cpu().numpy())])


image 1/1 /kaggle/working/predimg/eaa9119a-ce5b-11ee-8061-02420a00ab03.jpg: 640x640 6 0.88, 1 0.12, 2 0.00, 3 0.00, 4 0.00, 16.1ms
Speed: 21.2ms preprocess, 16.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/predimg/90e33e38-8aea-11ee-bfc8-02420a00aa08.jpg: 640x640 6 0.88, 5 0.08, 2 0.03, 3 0.01, 1 0.00, 16.1ms
Speed: 14.7ms preprocess, 16.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/predimg/f3ff1fb2-d623-11ee-83cd-02420a00ab03.jpg: 640x640 1 1.00, 6 0.00, 3 0.00, 4 0.00, 2 0.00, 16.0ms
Speed: 16.9ms preprocess, 16.0ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/predimg/4F9698F6-C0F5-4DE7-BF8B-AD2471CA60FF.jpg: 640x640 2 0.73, 4 0.26, 6 0.00, 1 0.00, 3 0.00, 16.0ms
Speed: 14.4ms preprocess, 16.0ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/predimg/66270abe-8f8e-11ee-81ee-02420a00aa08.jpg: 640

In [15]:
pred_df = pd.DataFrame({'id': ids, 'category': classpred})

In [16]:
pred_df.to_csv('urban_issues_submission.csv', index=False)